In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [22]:
import requests

# Replace with your eBay App's credentials
client_id = 'AaronPic-testing-PRD-68b654055-4e67ae71'
client_secret = 'PRD-5ada24c3dfc8-5c11-42a9-b1ba-1d34'

def get_ebay_oauth_token():
    url = "https://api.ebay.com/identity/v1/oauth2/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials",
        "scope": "https://api.ebay.com/oauth/api_scope"
    }
    response = requests.post(url, headers=headers, data=data, auth=(client_id, client_secret))
    response.raise_for_status()
    return response.json()["access_token"]

token = get_ebay_oauth_token()
print("Generated OAuth Token:", token)


Generated OAuth Token: v^1.1#i^1#I^3#p^1#r^0#f^0#t^H4sIAAAAAAAAAOVYfWwTZRhfuw1cYFsQFEOAlAOMYdz1PtvrhXaUrWSVfZS1G0gkeL17u912vbvcvUdXjLGMhMh/xviN0YkBxI+EBIJiFI2KCDF+gJIgGBRilIAfmIFB8OPuVkY3CV9rYhP7T3vP+7zP+/x+z+9537eH58ZVzVvftP73atd490AOz7ldLmICXjWusq6m3D2tsgwvcHAN5ObkKvrLf1xg8GlZ49qBoamKATx9aVkxOMcYRExd4VTekAxO4dPA4KDAxcMtzRyJ4Zymq1AVVBnxRBuDCOsjk6yfZQM0SQPa77esyuWYCTWIgABOAzJFpASCZ/2MPW4YJogqBuQVGERInKRRgkQJPEFQHENwDIPhAXoF4ukEuiGpiuWC4UjISZdz5uoFuV47Vd4wgA6tIEgoGl4cbwtHGyOtiQXeglihPA9xyEPTGPnUoIrA08nLJrj2MobjzcVNQQCGgXhDQyuMDMqFLydzC+k7VPM0y4hkMkCJPusHKAqTi1U9zcNrp2FbJBFNOa4cUKAEs9cj1CIj2QMEmH9qtUJEGz3211KTl6WUBPQgElkUvi8ciyGhMK+rSkwSUAgMKCldaKy9EfWxSR9D4wyD0sDn54GfyK8zFCxP8qiFGlRFlGzKDE+rChcBK2kwmhq6gBrLqU1p08MpaCdU6MdcppBlV9glHaqhCbsVu6ogbfHgcR6vX4Dh2RDqUtKEYDjC6AGHIavSmiaJyOhBR4l58fQZQaQbQo3zejOZDJahMFXv8pI4TniXtzTHhW6Q5hHH1+5121+6/gRUcqAIlrQsfw5mNSuXPkupVgJKFxKy9EfRdJ73kWmFRlv/ZSjA7B3ZD8XqD1LgqYBPBKLPFyDFQFEaJJTXqNfOAyT5LJrm9V4ANZkXACpYOjPTQJdEjmJSJMWmACr6AimUDqRSaJIRfSiRAgAHIJkUA

In [19]:
import pandas as pd

def search_laptops_in_uk(oauth_token, query="laptop", max_results=200):
    url = "https://api.ebay.com/buy/browse/v1/item_summary/search"
    headers = {
        "Authorization": f"Bearer {oauth_token}",
        "Content-Type": "application/json",
        "X-EBAY-C-MARKETPLACE-ID": "EBAY_GB"
    }
    all_items = []
    limit = 50  # Maximum items per request (can increase to 200)
    
    for offset in range(0, max_results, limit):
        # Add pagination parameters
        params = {
            "q": query,
            "filter": "itemLocationCountry:GB,priceCurrency:GBP",
            "limit": limit,
            "offset": offset
        }
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()

        # Append the new items
        items = data.get("itemSummaries", [])
        all_items.extend(items)

        # Stop if no more items are returned
        if len(items) < limit:
            break

    return all_items

# Fetch laptops
laptops = search_laptops_in_uk(token, max_results=10000)  # Get up to 500 results

# Process and display results
data = []
for item in laptops:
    title = item.get("title", "N/A")
    price = item.get("price", {}).get("value", "N/A")
    currency = item.get("price", {}).get("currency", "N/A")
    url = item.get("itemWebUrl", "N/A")
    location = item.get("itemLocation", {}).get("country", "N/A")
    data.append({"Title": title, "Price": price, "Currency": currency, "Location": location, "URL": url})

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

df.to_csv("ebaylistings.csv", index=False)


                                                  Title   Price Currency  \
0     HP Pro X2 Tablet 612 G2 i5-7th Gen 8GB 256GB S...  199.99      GBP   
1     Lenovo ThinkPad Windows 11 , 240GB SSD, Intel ...  109.99      GBP   
2     FAST WINDOWS 11 CHEAP LAPTOP INTEL CORE 16GB R...  179.99      GBP   
3     Laptop HP Windows 11 CORE i5 GEN 6 - 12TH GEN ...  159.99      GBP   
4     FAST WINDOWS 11 CHEAP LAPTOP INTEL CORE 8GB RA...   79.99      GBP   
...                                                 ...     ...      ...   
9995               Asus Chromebook C423N 14" Chromebook  119.99      GBP   
9996                          HP Elitebook 4880p - Used   60.00      GBP   
9997  BMAX Plus Intel Laptop 14" Windows 11 8GB RAM ...  399.00      GBP   
9998  Lenovo ThinkPad P53s 15.6" Laptop I7-8565U 16G...  249.99      GBP   
9999  PANASONIC TOUGHBOOK CF-31 BLACK CORE i5 MK1 TO...  699.99      GBP   

     Location                                                URL  
0          GB  https

In [23]:
def get_item_details(oauth_token, item_id):
    url = f"https://api.ebay.com/buy/browse/v1/item/{item_id}"
    headers = {
        "Authorization": f"Bearer {oauth_token}",
        "Content-Type": "application/json",
        "X-EBAY-C-MARKETPLACE-ID": "EBAY_GB"  # For UK marketplace

    }
    
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

# Fetch details for a specific item
item_id = "126785583606"  # Replace with an actual item ID
item_details = get_item_details(token, item_id)

# Extract item specifics
localized_aspects = item_details.get("localizedAspects", [])
for aspect in localized_aspects:
    print(f"{aspect['name']}: {aspect['value']}")


HTTPError: 404 Client Error: Not Found for url: https://api.ebay.com/buy/browse/v1/item/126785583606